In [1]:
import tweepy
import numpy
import pandas as pd
import requests
from bs4 import BeautifulSoup
import importlib
import json
from helper import *
from selenium import webdriver
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn import linear_model

In [2]:
pd.options.display.max_columns = 200

In [3]:
df=pd.read_csv("out.csv")
df=pd.concat([df,pd.read_csv("out2.csv"),pd.read_csv("out3.csv"),pd.read_csv("out4.csv"),pd.read_csv("out5.csv"),pd.read_csv("out6.csv")][:3])
df.head()

,Unnamed: 0,name,age,city,country,gender,acount age,total tweets,followers,following,respectability,verified,text,views,likes,retweets,quote retweets,comments,word count,is quote,hashtags,hashtag count,trending,using trends?,link to tweet
0,0,CamuyArenas_AA,NaN,Puente,United States,NaN,3.2,228,50,22,0.440000,False,Just posted a photo @ Estadio Juan F.Cheo Lópe...,13,0,0,0,0,11,False,NaN,0,"['Friday the 13th', 'Staley', 'Lori Harvey', '...",False,https://www.twitter.com/CamuyArenas_AA/status/...
1,1,16p11dot2ds,NaN,Georgia,United States,NaN,4.9,32312,33,481,14.575758,False,This is a prep free zone. https://t.co/2qqlLxy...,336,1,0,0,1,8,True,NaN,0,"['Friday the 13th', 'Staley', 'Lori Harvey', '...",False,https://www.twitter.com/16p11dot2ds/status/161...
2,2,PoopScoopSF,NaN,Downtown,United States,NaN,4.7,163626,415,125,0.301205,False,"""Big poop front of our door."" (Downtown, D3) h...",9,0,0,0,1,10,False,NaN,0,"['Friday the 13th', 'Staley', 'Lori Harvey', '...",False,https://www.twitter.com/PoopScoopSF/status/161...
3,3,wchazz,NaN,Wesley Chapel,United States,NaN,15.4,48876,377,951,2.522546,False,"2nd cardio, target calories 1k @ Wesley Chapel...",4,0,0,0,0,10,False,NaN,0,"['Friday the 13th', 'Staley', 'Lori Harvey', '...",False,https://www.twitter.com/wchazz/status/16139885...
4,4,wyatthebson,NaN,Pier 17,United States,NaN,16.8,24061,1366,4999,3.659590,False,Listen to PODCAST #6 SGUZR by Core Beats on Tn...,13,0,0,0,0,11,False,NaN,0,"['Friday the 13th', 'Staley', 'Lori Harvey', '...",False,https://www.twitter.com/wyatthebson/status/161...


In [ ]:
df=df.drop(["age", "gender",'hashtags','Unnamed: 0'], axis=1)

In [ ]:
df.insert(df.columns.get_loc("respectability")+1,'is respactable?' ,[1 if x >= .75 else 0 for x in df['respectability']])

In [ ]:
df=df.drop_duplicates(subset='text',keep=False)

In [45]:
dups = df[df['name'].duplicated(keep = False)]['name'].value_counts()

for n in dups.keys():
    if dups[n] > 2:
        df.drop(df[df['name'] == n].index,inplace = True)
        
df[df['name'].duplicated()]['name'].value_counts().max()

1

In [79]:
df=df.drop(["link to tweet"], axis=1)

In [80]:
column_to_move = df.pop("retweets")

In [81]:
df.insert(df.shape[1], "retweets", column_to_move)